In [18]:
import autograd
import autograd.numpy as np
import autograd.scipy as sp
import paragami
import scipy as osp

In [2]:
np.random.seed(42)

num_obs = 1000

# True values of parameters
true_sigma = \
    np.eye(3) * np.diag(np.array([1, 2, 3])) + \
    np.random.random((3, 3)) * 0.1
true_sigma = 0.5 * (true_sigma + true_sigma.T)

true_mu = np.array([0, 1, 2])

# Data
x = np.random.multivariate_normal(
    mean=true_mu, cov=true_sigma, size=(num_obs, ))

# Original weights.
original_weights = np.ones(num_obs)

In [41]:
def regularizer(par_dict, lam):
    return lam * np.sum(par_dict['mu'] ** 2)

def model_logpdf(par_dict, weights, x):
    data_lpdf = sp.stats.multivariate_normal.logpdf(
        x, mean=par_dict['mu'], cov=par_dict['sigma'])
    return np.sum(weights * data_lpdf)

def objective(par_dict, weights, lam, x):
    return -model_logpdf(par_dict, weights, x) + regularizer(par_dict, lam)

norm_par = dict()
norm_par['mu'] = true_mu
norm_par['sigma'] = true_sigma

original_lam = 1

objective(norm_par, original_weights, original_lam, x)

5154.567522214259

In [43]:
norm_param_pattern = paragami.PatternDict()
norm_param_pattern['sigma'] = paragami.PSDSymmetricMatrixPattern(size=3)
norm_param_pattern['mu'] = paragami.NumericArrayPattern(shape=(3, ))

objective_flat = paragami.FlattenFunctionInput(
    lambda par_dict: objective(par_dict, original_weights, original_lam, x),
    patterns=norm_param_pattern,
    free=True)

assert(
    objective_flat(norm_param_pattern.flatten(par_dict, free=True)) == \
    objective(norm_par, original_weights, lam, x))

In [ ]:
#objective_flat_par = paragami.